In [ ]:
#Test the effect of fraction of small cells that also can participate in the signal relay

import numpy as np                                                                                                                 
import matplotlib.pyplot as plt
import math
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
#Define functions used for diffusion partial differential equations

#Define Laplace operator with zero-flux boundary condition
def dd2 (A,x,y,dx,dy,i,j):
  if i==0:
    dAx2=(A[i+1,j]-2*A[i,j]+A[i+1,j])/(dx*dx)
  elif i==x-1:
    dAx2=(A[i-1,j]-2*A[i,j]+A[i-1,j])/(dx*dx)
  else:
    dAx2=(A[i+1,j]-2*A[i,j]+A[i-1,j])/(dx*dx)

  if j==0:
    dAy2=(A[i,j+1]-2*A[i,j]+A[i,j+1])/(dy*dy)
  elif j==y-1:
    dAy2=(A[i,j-1]-2*A[i,j]+A[i,j-1])/(dy*dy)
  else:
    dAy2=(A[i,j+1]-2*A[i,j]+A[i,j-1])/(dy*dy)
    
  d=dAx2+dAy2

  return d;

#Define Erk propagation function
#E0, initial condition
#Cell id, cells position
#T, total simulation time
#dt, simulation time step
#c_threhold, activition threshold
#cell_len, cell length
#activition rate, a
#Diffusion coefficient, D

def Erk_Propogation_2D (E0,Cell_id,num_grid_x,num_grid_y,grid_size,T,dt,step,c_threhold,cell_len,a):  
   t1=0
   t2=0
   E=E0.copy()
   v=0
   L=np.zeros((step-1,num_grid_x))
   for time in range(step-1): 
     Es=E
     for i in range(num_grid_x):
       for j in range(num_grid_y):
         if i<10:
           E[i,j]=Es[i,j]+dt*(D*dd2(Es,num_grid_x,num_grid_y,grid_size,grid_size,i,j)+a*(1-Es[i,j]))
         else:
           E[i,j]=Es[i,j]+dt*(D*dd2(Es,num_grid_x,num_grid_y,grid_size,grid_size,i,j)+a*theta_n(Es,Cell_id,i,j,c_threhold,cell_len))
     if t1==0:
       if np.min(E[11,:])>0.1:
         t1=time
     if t2==0:
       if np.min(E[num_grid_x-5,:])>0.1:
         t2=time
         v=((num_grid_x-16)*cell_len*grid_size)/((t2-t1)*dt)
     else:
        break
     L[time,:]=np.mean(E, axis=1).T
       
   return E,v,L;

#Define Heaviside step function

def theta_n (E,Cell_id,i,j,c_threhold,cell_len): 
  c=0
  if Cell_id[i,j]==0:
    c=0
  elif Cell_id[i,j]==-1:
    if E[i,j]>c_threhold:
        c=1-E[i,j]
  else:
    if max(E[int(i-Cell_id[i,j]+1):min(int(i-Cell_id[i,j]+1+cell_len),num_grid_x),j])>c_threhold:
        c=1-E[i,j]
  return c;

#Generate cell pattern for signal propagation with given parameters
#num_grid_x, num_grid_y, number of cells on x- and y-axis
#bias, distribution bias of cells
#den_volum, volume density of long cells
#den_small, volume density of small relay cells
#cell_len, long cell length

def Generate_cell_id (num_grid_x,num_grid_y,bias,den_volum,den_small,cell_len): 
  Cell_id=np.zeros((num_grid_x,num_grid_y))-1
  dev_h=den_volum+bias
  dev_l=den_volum-bias
  density_h=dev_h/(cell_len+dev_h-cell_len*dev_h)
  density_l=dev_l/(cell_len+dev_l-cell_len*dev_l)
  for j in range(num_grid_y):
    i=0
    while i<num_grid_x:
      indic=np.random.uniform()
      if j<0.5*num_grid_y:
        den= density_h
      else:
        den= density_l
      if indic>den:
        indic_small=np.random.uniform()
        if indic_small>indic_small:
            Cell_id[i,j]=0
            i=i+1
        else:
            Cell_id[i,j]=-1
            i=i+1
      else:
        d=min(cell_len,num_grid_x-i)
        Cell_id[i:i+d,j]=np.linspace(1,d,num=d)
        i=i+cell_len

  return Cell_id;

#data extraction plotting speed over long cell length for data structure defined

def data_plot(c_len,v):
    #0den #1len #2dis $3speed
    xdata=np.zeros(0) #den
    ydata=np.zeros(0) #len
    z1data=np.zeros(0) #dis
    z2data=np.zeros(0) #speed
    for i in range(len(v[0,:])):
        if v[1,i]==c_len:
            xdata=np.append(xdata,v[0,i])
            ydata=np.append(ydata,v[1,i])
            z1data=np.append(z1data,v[2,i])
            z2data=np.append(z2data,v[3,i])
    
    return xdata, ydata, z1data, z2data

#data extraction plotting speed over small relay cell density for data structure defined

def data_plot_celldirec(c_len,v):
    #0den #1len #2dis $3speed
    xdata=np.zeros(0) #den
    ydata=np.zeros(0) #direc
    z1data=np.zeros(0) #dis
    z2data=np.zeros(0) #speed
    
    bis_space=np.linspace(0,1,11)
    speed=np.zeros((5,len(bis_space)))
    
    for i in range(len(v[0,:])):
        if (v[0,i])==c_len:
            xdata=np.append(xdata,v[0,i])
            ydata=np.append(ydata,v[1,i])
            z1data=np.append(z1data,v[2,i])
            z2data=np.append(z2data,v[3,i]*60/5)
            
#    print(z2data)
    for n in range(len(bis_space)):
        for m in range(5):
            speed[m,len(bis_space)-n-1]=z2data[n+m*len(bis_space)]
    
    return xdata, ydata, z1data, z2data, speed

In [ ]:
#Simulation parameters set up

T=500 #min

range_grid_x=1000
range_grid_y=200
grid_size=10
num_grid_x=int(range_grid_x/grid_size)
num_grid_y=int(range_grid_y/grid_size)
E0=np.zeros((num_grid_x,num_grid_y))

c_threhold=0.1
D=0.1*60
a=0.2
dt=0.01
step=int(T/dt)
bias=0

In [ ]:
repeat=5
c_len=int(5)
data=np.zeros((4,repeat*len(den_space)*len(den_small))) #0 den, 1 len, 2 dis, 3 speed
for r in range(repeat):
    print(r)
    for i in range(len(den_space)):
        for j in range(len(den_small)):
            E0=np.zeros((num_grid_x,num_grid_y))
            Cell_id=Generate_cell_id (num_grid_x,num_grid_y,bias,den_space[i],den_small[j],c_len)
            E,v,L2 = Erk_Propogation_2D (E0,Cell_id,num_grid_x,num_grid_y,grid_size,T,dt,step,c_threhold,c_len,a)
            index=int(r*len(den_space)*len(den_small)+i*len(den_small)+j)
            data[0,index]=den_space[i]
            data[1,index]=den_small[j]
            data[2,index]=Find_most_distance(Cell_id)-9
            data[3,index]=v
            print(r,i,j,index/(repeat*len(den_space)*len(den_small)))
        
np.savetxt('erk_simulation_data_smallcell.csv', data, delimiter=',')
v=np.loadtxt('erk_simulation_data_smallcell.csv', delimiter=',')

In [ ]:
#plot wave speed over long cell length with different small relay cell densities

c1='#EF6145'
c2='#389ADD'
c3='#45A39B'
c4='#E03960'

plt.figure(figsize=(4, 3.5))

c_len=0.9
xdata, ydata, z1data, z2data, speed = data_plot_n(c_len,v)
plt.scatter(xdata,z2data,color=c1,s=20,alpha=0.7)
speed_mean=np.mean(speed,axis=0)
speed_error=np.std(speed,axis=0)
plt.errorbar(den_space,speed_mean,yerr=speed_error,ecolor=c1,color=c1,elinewidth=2,capsize=4,capthick=2,alpha=0.7)


c_len=v[1,3]
xdata, ydata, z1data, z2data, speed = data_plot_n(c_len,v)
plt.scatter(xdata,z2data,color=c2,s=20,alpha=0.7)
speed_mean=np.mean(speed,axis=0)
speed_error=np.std(speed,axis=0)
plt.errorbar(den_space,speed_mean,yerr=speed_error,ecolor=c2,color=c2,elinewidth=2,capsize=4,capthick=2,alpha=0.7)


c_len=0
xdata, ydata, z1data, z2data, speed = data_plot_n(c_len,v)
plt.scatter(xdata,z2data,color=c3,s=20,alpha=0.7)
speed_mean=np.mean(speed,axis=0)
speed_error=np.std(speed,axis=0)
plt.errorbar(den_space,speed_mean,yerr=speed_error,ecolor=c3,color=c3,elinewidth=2,capsize=4,capthick=2,alpha=0.7)

In [ ]:
#plot wave speed over different small relay cell densities

c1='#EF6145'
c2='#389ADD'
c3='#45A39B'

plt.figure(figsize=(4, 3.5))
plt.ylim([-30,550])

den_space=np.linspace(0,1,21)
den_small=np.linspace(0,1,11)

bia_space=np.linspace(0,1,11)
print(bia_space)
print(den_space[6])

c_len=den_space[6]
xdata, ydata, z1data, z2data, speed = data_plot_celldirec(c_len,v)
plt.scatter(ydata,z2data,color=c3,s=20,alpha=0.7)
speed_mean=np.mean(speed,axis=0)
speed_error=np.std(speed,axis=0)
plt.errorbar(1-bia_space,speed_mean,yerr=speed_error,ecolor=c3,color=c3,elinewidth=2,capsize=4,capthick=2,alpha=0.7)

c_len=0.5
xdata, ydata, z1data, z2data, speed = data_plot_celldirec(c_len,v)
plt.scatter(ydata,z2data,color=c2,s=20,alpha=0.7)
speed_mean=np.mean(speed,axis=0)
speed_error=np.std(speed,axis=0)
plt.errorbar(1-bia_space,speed_mean,yerr=speed_error,ecolor=c2,color=c2,elinewidth=2,capsize=4,capthick=2,alpha=0.7)

c_len=0.8
xdata, ydata, z1data, z2data, speed = data_plot_celldirec(c_len,v)
plt.scatter(ydata,z2data,color=c1,s=20,alpha=0.7)
speed_mean=np.mean(speed,axis=0)
speed_error=np.std(speed,axis=0)
plt.errorbar(1-bia_space,speed_mean,yerr=speed_error,ecolor=c1,color=c1,elinewidth=2,capsize=4,capthick=2,alpha=0.7)